In [1]:
import numpy as np 
import matplotlib.pyplot as plt
import sacc 
from scipy.stats import chi2 as chi2f
import pyccl as ccl 
from scipy.interpolate import interp1d
from matplotlib import rc
rc('font', **{'family': 'sans-serif', 'sans-serif': ['Helvetica']})
rc('text', usetex=True)

# Extracting F measurements from $C_{\ell}'s$

The aim of this notebook is to demonstrate how the F measurements used to constrain $\langle \sigma v \rangle$ and $\Gamma$ can be extracted from our cross-correlation measurements $C_{\ell}$. We follow the method outlined in Section II D 4 of the paper.
We will show how this is done for the case of decay. 

We begin by reading some data:

In [2]:
sa = sacc.Sacc.load_fits("anyas_fermi_x_galaxies_.fits")
sa.remove_selection(ell__gt=2048)
sa.remove_selection(ell__lt=30)

num_gal = 6
num_gam = 12

# Calculate number of ells
ells, _ = sa.get_ell_cl('cl_00', 'gal0', 'gam0')
num_ell = len(ells)

#Store the ells
store_ell = np.zeros(24)
for i,ell in enumerate(ells):
    val_ell =float('%.3f'%(ell))
    store_ell[i] = val_ell

# Calculate number of redshifts
zs = sa.tracers['gal5'].z
num_z = len(zs)

#Calculating the mean redshift of each bin
zmeans = []
for igal in range(num_gal):
    t = sa.tracers[f'gal{igal}']
    zmeans.append(np.average(t.z, weights=t.nz))

# Resample first redshift bin at the same redshifts as the rest
nz0_i = interp1d(sa.tracers['gal0'].z, sa.tracers['gal0'].nz, bounds_error=False, fill_value=0)
sa.tracers['gal0'].z = zs
sa.tracers['gal0'].nz = nz0_i(zs)

# Extract beams
goodl = [ell in store_ell for ell in sa.tracers['gam0'].ell]
beams = {f'gam{igam}': sa.tracers[f'gam{igam}'].beam[goodl]
         for igam in range(num_gam)}

/mnt/zfsusers/anyabua/.local/lib/python3.6/site-packages/sacc/tracers.py:44: UserWarning: Unknown quantity intensity_map. If possible use a pre-defined quantity, or add to the list.
  warnings.warn(f"Unknown quantity {quantity}. "


# 1. Then we create the window function:

Refer to Eq. 36 in the paper. 

The window function is given by:
    
$$W_{i,n}(z) \equiv\frac{1}{E_{i+1}-E_i}\int_{E_i(1+z)}^{E_{i+1}(1+z)}d\epsilon\,\Theta(\epsilon_n<\epsilon<\epsilon_{n+1})
$$

Which is equal to 

$$W_{i,n}(z) = \frac{\text{Min}[E_{i+1}(1+z),\epsilon_{n+1}]-\text{Max}[E_{i}(1+z),\epsilon_{n}]}{E_{i+1}-E_{i}}\times\Theta(\epsilon_{n+1}>E_{i}(1+z))\times\Theta(\epsilon_{n}<E_{i+1}(1+z)).$$

Here, the raw measurements of $C_{\ell}$ has units ${\rm cm}^{-2}s^{-1}$. So to match the units up, we multiply through by the energy:

$$W_{i,n}(z) = (\text{Min}[E_{i+1}(1+z),\epsilon_{n+1}]-\text{Max}[E_{i}(1+z),\epsilon_{n}])\times\Theta(\epsilon_{n+1}>E_{i}(1+z))\times\Theta(\epsilon_{n}<E_{i+1}(1+z)).$$

This will only affect the units we'd like F to be in. 

In [3]:
# Energy ranges in GeV
E_obs = np.array([5.24807460e+02, 1.00000000e+03, 1.73780083e+03, 3.01995172e+03,
                  5.24807460e+03, 8.31763771e+03, 1.58489319e+04, 2.29086765e+04,
                  3.98107171e+04, 7.58577575e+04, 1.20226443e+05, 3.31131121e+05,
                  1.00000000e+06])*1E-3
# Rest-frame
E_emt = E_obs[:, None]*(1+zs[None, :])
num_E = len(E_obs)-1
assert num_E == num_gam
windows = np.zeros([num_E, num_gam, num_z])

for n in range(num_E):
    for i in range(num_gam):
        E_n = E_obs[n]
        E_i = E_obs[i]
        E_i_oneplusz = E_emt[i]
        E_np = E_obs[n+1]
        E_ip = E_obs[i+1]
        E_ip_oneplusz = E_emt[i+1]
        Emin = np.minimum(E_ip_oneplusz, E_np)
        Emax = np.maximum(E_i_oneplusz, E_n)
        good_n = E_ip_oneplusz > E_n
        good_np = E_i_oneplusz < E_np
        win = (Emin-Emax)*good_n*good_np
        windows[n, i, :] = win

In [4]:
#Units are in GeV
Eeff = np.array([  0.70608114, 1.29359873, 2.24801694, 3.9066057, 6.52075968, 11.19063184, 
                 18.89501006, 29.63463478, 53.5617316,94.25365366, 187.36854987,534.05806527])

# 2. Get $P(k)$ and theoretical $C_{\ell}$ templates

First we get the decay profile: $(1 + \delta)$. This is simply the NFW profile contained in pyccl. 

In [6]:
cosmo = ccl.Cosmology(Omega_c=0.27, Omega_b=0.045, h=0.67, sigma8=0.83, n_s=0.96) #need to make a K(z) function
k_arr = np.geomspace(1e-4,100,256)
a = 1./(1+zs)
a_arr = a[::-1]
chi = ccl.comoving_radial_distance(cosmo,a_arr)[::-1]

# We will use a mass definition with Delta = 200 times the critical density
hmd_200c = ccl.halos.MassDef200c() 
# The Duffy 2008 concentration-mass relation
cM = ccl.halos.ConcentrationDuffy08(hmd_200c)
# The Tinker 2008 mass function
nM = ccl.halos.MassFuncTinker08(cosmo, mass_def=hmd_200c)
# The Tinker 2010 halo bias
bM = ccl.halos.HaloBiasTinker10(cosmo, mass_def=hmd_200c)
# The NFW profile to characterize the matter density around halos
pM = ccl.halos.profiles.HaloProfileNFW(cM)
# Halo model calculator
hmc = ccl.halos.HMCalculator(cosmo, nM, bM, hmd_200c)
pk_MDM = ccl.halos.halomod_Pk2D(cosmo, hmc, pM, prof2=pM,
                                 normprof1=True, normprof2=True,
                                 lk_arr=np.log(k_arr), a_arr=a_arr)

bias = np.array([1.182,1.086,1.126,1.144,1.206,1.548])

# This is density in M_sun/Mpc^3
rho_x = ccl.background.rho_x(cosmo, 1, 'matter',
                             is_comoving = True)
C = rho_x/(4*np.pi*(1+zs))

tgal = []
for gal in range(0,6):
    tr = sa.tracers[f'gal{gal}']
    t = ccl.NumberCountsTracer(cosmo, has_rsd = False,
                               dndz=(tr.z, tr.nz),
                               bias=(tr.z, np.full_like(tr.z, bias[gal])))
    tgal.append(t)

# C_ell will have units of M_sun*GeV/Mpc^2
cl_th = np.zeros([num_gal, num_E, num_gam, num_ell])
for ig, g in enumerate(tgal):
    for n in range(num_E):
        for i in range(num_gam):
            GRB_tracer = ccl.Tracer()
            GRB_tracer.add_tracer(cosmo, kernel=(chi, C*windows[n, i]))
            cl_theoretical = ccl.angular_cl(cosmo, g, GRB_tracer, ells, p_of_k_a= pk_MDM)
            cl_th[ig, n, i, :] = cl_theoretical*beams[f'gam{i}']
            


# 3. Measure F!
For this, we do:
$$
{\bf F}={\sf C}_F\,{\bf A}
$$
With
$$
({\sf C}_F^{-1})_{mn}=\sum_{gi\ell}\sum_{g'i'\ell'}C_\ell^{g,in}({\rm Cov}^{-1})_{(gi\ell),(g'i'\ell')}C_{\ell'}^{g',i'm}
$$
and
$$
A_n=\sum_{gi\ell}\sum_{g'i'\ell'}C_\ell^{g,in}({\rm Cov}^{-1})_{(gi\ell),(g'i'\ell')}\hat{C}_{\ell'}^{g'i'}
$$
Note that ${\sf C}_F$ also happens to be the covariance of ${\bf F}$.

Here $\hat{C}_\ell^{gi}$ is the data $g-i$ cross-correlation, and $C^{g,in}_\ell$ is the theoretical template for galaxy sample $g$ and window function $W_{n,i}(z)$.

At the end we multiply the result by $8.53323018\times10^{-9}$ to convert $F$ from ${\rm cm}^{-1}{\rm s}^{-1}{\rm Mpc}^2 M_\odot^{-1}\,{\rm GeV}$ to ${\rm s}^{-1}\,{\rm GeV}^{-2}$.

In [17]:
def get_F(gals):
    prefac = 8.53323018E-9
    ngal = len(gals)
    gams = range(num_gam)

    # Get indices
    indices = []
    cl_data = []
    for igal, gal in enumerate(gals):
        indices.append([])
        cl_data.append([])
        for gam in gams:
            _, cl, ind = sa.get_ell_cl('cl_00', f'gal{gal}', f'gam{gam}', return_ind=True)
            indices[igal].append(ind)
            cl_data[igal].append(cl)
    indices = np.array(indices)
    cl_data = np.array(cl_data)

    # Get theory C_ells
    # Shape: [ngal, n_n, n_i, ell]
    cl_theory = np.array([cl_th[ig] for ig in gals])
    # Shape: [n_n, ngal, n_i, ell]
    cl_theory = np.transpose(cl_theory, axes=[1, 0, 2, 3])

    # Flatten all relevant dimensions
    cl_data = cl_data.flatten()
    indices = indices.flatten()
    cl_theory = cl_theory.reshape([num_E, -1])
    
    # Construct covariance
    cv = sa.covariance.covmat.copy()
    cv = cv[indices][:, indices]

    # Q matrix
    # Cov^-1 C_th
    iC_th = np.array([np.linalg.solve(cv, th)
                      for th in cl_theory])
    # C_th^T Cov^-1 C_th
    Q = np.sum(cl_theory[:, None, :]*iC_th[None, :, :], axis=-1)

    # A vector
    # C_th^T Cov^-1 d
    A = np.dot(cl_theory, np.linalg.solve(cv, cl_data))

    # F = Q^-1 F
    cov_F = np.linalg.inv(Q)
    F = np.dot(cov_F, A)
    
    # Prefactors
    F *= prefac
    cov_F *= prefac**2

    return F, cov_F

We can now get the F's and store them.

In [18]:
res_F = {f'gal{i}': get_F([i]) for i in range(6)}
res_F ['total'] = get_F(range(6))

# Saving

In [19]:
F_units = ['cm^-3 GeV^-3 s^-1']
Energy_units = ['GeV']

Fsave = np.array([v[0] for k, v in res_F.items()])
covFsave = np.array([v[1] for k, v in res_F.items()])
names = [k for k, v in res_F.items()]
np.savez("F_decay.npz", names=names, F=Fsave, covF=covFsave, Eeff = Eeff, z_mean = zmeans, F_units = F_units,
        Energy_units = Energy_units)